In [69]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [ ]:
### importing holidays/off dates for any district in Austin during the 2023-24 school year
    ## this file should include any weekday that a district has no school (True), including the dates prior to the first day of school
offDates = pd.read_csv('austinOffDates_schoolYear_23_24.csv').drop(columns=['Holiday'])
offDates

,Date,Federal_Holiday,Austin,Round_Rock,Del_Valle,Manor,Lake_Travis,Eanes,Leander
0,2023-08-01,False,True,True,True,True,True,True,True
1,2023-08-02,False,True,True,True,True,True,True,True
2,2023-08-03,False,True,True,True,True,True,True,True
3,2023-08-04,False,True,True,True,True,True,True,True
4,2023-08-07,False,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...
66,2024-05-27,True,True,True,True,True,True,True,True
67,2024-05-28,False,True,True,True,False,True,True,True
68,2024-05-29,False,True,True,True,False,True,True,True
69,2024-05-30,False,True,True,True,False,True,True,True


In [71]:
### define the beginning and ending dates for the span for modeling
first_date = datetime(year=2023, month=8, day=1)
last_date = datetime(year=2024, month=5, day=31)
date_range = (last_date - first_date)

In [72]:
### create list of all dates in the date range using a generator expression
date_list_school = [ last_date - timedelta(days=x) for x in range(date_range.days+1) ]

In [73]:
### create dictionary that indicates whether the date is a weekday(True) or weekend(False)
### python designates datetime.weekday() == 5 as Saturday and datetime.weekday() == 6 as Sunday
weekday = {i: (True if i.weekday() < 5 else False) for i in date_list_school}

In [74]:
### use weekday dictionary to create dataframe and transpose
weekday_df = pd.DataFrame.from_dict([weekday]).transpose().reset_index()
weekday_df.columns = ['Date', 'is_weekday']

In [75]:
### make sure that the Date columns for the weekday and holidays dataframes are of the type datetime64[ns]
weekday_df['Date'] = weekday_df['Date'].astype('datetime64[ns]')
offDates['Date'] = offDates['Date'].astype('datetime64[ns]')

In [76]:
### create dictionary for percentage of city covered by each district, unique to each city
districtPercents = {'Austin': 0.67, 'Round_Rock': 0.11, 'Del_Valle': 0.07, 'Manor': 0.05, 'Lake_Travis': 0.04, 'Eanes': 0.03, 'Leander': 0.03}

In [77]:
def calculate_school_day_percentage(district_percentages):
    """
    Calculates the percentage of each potential off date based on all school districts and creates the 'is_school_day' column in the offDates date frame

    Args:
        district_percentages (dict): keys (str) are the districts matching the column names in the offDates dateFrame 
                                        values (float) are the city coverage percentages
    """
    
    ## if district does have the day off(True) replace with 0, else(False) replace with district percentage
    for key, value in district_percentages.items():
        offDates[key] = offDates[key].replace({True: 0, False: value})

    ## creates 'is_school_day' column by summing over the district columns
    ## sum will reflect percentage of city that has the school day off
    offDates['is_school_day'] = offDates.drop(columns=['Date', 'Federal_Holiday']).sum(axis=1)

In [ ]:
districtPercents = {'Austin': 0.67, 'Round_Rock': 0.11, 'Del_Valle': 0.07, 'Manor': 0.05, 'Lake_Travis': 0.04, 'Eanes': 0.03, 'Leander': 0.03}
calculate_school_day_percentage(districtPercents)

In [79]:
offDates.head(20)

,Date,Federal_Holiday,Austin,Round_Rock,Del_Valle,Manor,Lake_Travis,Eanes,Leander,is_school_day
0,2023-08-01,False,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2023-08-02,False,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2023-08-03,False,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2023-08-04,False,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2023-08-07,False,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,2023-08-08,False,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,2023-08-09,False,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.07
7,2023-08-10,False,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.07
8,2023-08-11,False,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.07
9,2023-08-14,False,0.67,0.00,0.07,0.05,0.00,0.00,0.00,0.79


In [80]:
districts = ['Austin', 'Round_Rock', 'Del_Valle', 'Manor', 'Lake_Travis', 'Eanes', 'Leander']

In [81]:
### combine the weekday and holiday dataframes by matching the dates
    ## dates not in the holiday dataframe will have NaN, so replace with False for 'Federal_Holiday'
    ## keep NaN for 'is_school_day' in order to properly using conditional logic later on
schoolWork_schedule = pd.merge(weekday_df, offDates.drop(columns=districts), on='Date', how='left')
schoolWork_schedule['Federal_Holiday'].fillna(False, inplace=True)
schoolWork_schedule.head()

,Date,is_weekday,Federal_Holiday,is_school_day
0,2024-05-31,True,False,0.05
1,2024-05-30,True,False,0.05
2,2024-05-29,True,False,0.05
3,2024-05-28,True,False,0.05
4,2024-05-27,True,True,0.00


In [82]:
schoolWork_schedule.head(20)

,Date,is_weekday,Federal_Holiday,is_school_day
0,2024-05-31,True,False,0.05
1,2024-05-30,True,False,0.05
2,2024-05-29,True,False,0.05
3,2024-05-28,True,False,0.05
4,2024-05-27,True,True,0.00
5,2024-05-26,False,False,NaN
6,2024-05-25,False,False,NaN
7,2024-05-24,True,False,0.10
8,2024-05-23,True,False,NaN
9,2024-05-22,True,False,NaN


In [83]:
schoolDay = schoolWork_schedule['is_school_day']
weekDay = schoolWork_schedule['is_weekday']
fedHoliday = schoolWork_schedule['Federal_Holiday']
dateC = schoolWork_schedule['Date']

In [84]:
### matching the weekday/weekend conditions to the non-holiday dates in the is_school_day column
    ## if a weekday and not a federal holiday then replace NaN with 1, else if weekend replace NaN with 0
    ## if non-False entry in 'is_school_day' keep the entry
schoolWork_schedule['is_school_day'] = np.where( (schoolDay.isna()) & (weekDay == False) | (fedHoliday == True), 0.0,  schoolWork_schedule['is_school_day'])
schoolWork_schedule['is_school_day'] = np.where( (schoolDay.isna()) & (weekDay == True) & (fedHoliday == False), 1.0,  schoolWork_schedule['is_school_day'])

In [85]:
### check that weekday/weekend matching logic is correct, look at dates around Thanksgiving break for reference
    ## Thanksgiving dates are chosen arbitarily, can use to check any dates to verify logic is applied correctly
schoolWork_schedule.loc[(dateC > datetime(year=2023, month=11, day=12)) & (dateC < datetime(year=2023, month=11, day=28))]

,Date,is_weekday,Federal_Holiday,is_school_day
186,2023-11-27,True,False,1.00
187,2023-11-26,False,False,0.00
188,2023-11-25,False,False,0.00
189,2023-11-24,True,False,0.00
190,2023-11-23,True,True,0.00
191,2023-11-22,True,False,0.00
192,2023-11-21,True,False,0.00
193,2023-11-20,True,False,0.00
194,2023-11-19,False,False,0.00
195,2023-11-18,False,False,0.00


In [86]:
### checking first day of school per district 
schoolWork_schedule.tail(20)

,Date,is_weekday,Federal_Holiday,is_school_day
285,2023-08-20,False,False,0.00
286,2023-08-19,False,False,0.00
287,2023-08-18,True,False,1.00
288,2023-08-17,True,False,1.00
289,2023-08-16,True,False,1.00
290,2023-08-15,True,False,0.79
291,2023-08-14,True,False,0.79
292,2023-08-13,False,False,0.00
293,2023-08-12,False,False,0.00
294,2023-08-11,True,False,0.07


In [87]:
### checking last day of school per district 
schoolWork_schedule.head(20)

,Date,is_weekday,Federal_Holiday,is_school_day
0,2024-05-31,True,False,0.05
1,2024-05-30,True,False,0.05
2,2024-05-29,True,False,0.05
3,2024-05-28,True,False,0.05
4,2024-05-27,True,True,0.00
5,2024-05-26,False,False,0.00
6,2024-05-25,False,False,0.00
7,2024-05-24,True,False,0.10
8,2024-05-23,True,False,1.00
9,2024-05-22,True,False,1.00


In [88]:
### create work day column based on weekday and federal holiday columns
    ## is_work_day entry is 0 if the date is a weekend or federal holiday
schoolWork_schedule['is_work_day'] = [
    0 if (w == False) or (f == True) else 1 for w, f in zip(schoolWork_schedule['is_weekday'], schoolWork_schedule['Federal_Holiday'])
]

In [89]:
### checking that the entries for is_school_day and is_work_day columns follow the logic based on the other columns
schoolWork_schedule.sort_values('Date').tail(30)

,Date,is_weekday,Federal_Holiday,is_school_day,is_work_day
29,2024-05-02,True,False,1.00,1
28,2024-05-03,True,False,1.00,1
27,2024-05-04,False,False,0.00,0
26,2024-05-05,False,False,0.00,0
25,2024-05-06,True,False,1.00,1
24,2024-05-07,True,False,1.00,1
23,2024-05-08,True,False,1.00,1
22,2024-05-09,True,False,1.00,1
21,2024-05-10,True,False,1.00,1
20,2024-05-11,False,False,0.00,0


In [90]:
### only keeping the date, is_school_day and is_work_day columns for the calender dataframe
school_work_calender = schoolWork_schedule.drop(columns=['is_weekday', 'Federal_Holiday']).rename(columns={'Date': 'date'})

In [91]:
### sorting the calender in chronological order
### resetting index so that index column follows chronological dates as well
school_work_calender_Seq = school_work_calender.sort_values('date').reset_index()

In [92]:
### convert dates to strings of YYYY-MM-DD format that is neccessary for the CLT model input
school_work_calender_Seq['date'] = school_work_calender_Seq['date'].dt.strftime('%Y-%m-%d')

In [93]:
### removing the unneccassary 'index' column
school_work_calender_Seq.drop(columns='index', inplace=True)
school_work_calender_Seq

,date,is_school_day,is_work_day
0,2023-08-01,0.00,1
1,2023-08-02,0.00,1
2,2023-08-03,0.00,1
3,2023-08-04,0.00,1
4,2023-08-05,0.00,0
...,...,...,...
300,2024-05-27,0.00,0
301,2024-05-28,0.05,1
302,2024-05-29,0.05,1
303,2024-05-30,0.05,1


In [94]:
school_work_calender_Seq.to_csv('school_work_calender_austinPercent_2023_24_schoolYear.csv')